In [1]:
from transformers import RobertaTokenizerFast, BertTokenizerFast, DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained('/mnt/transcriber/Call_Scoring/sa_tokenizer/', add_special_tokens=True)

In [2]:
from transformers import DistilBertConfig

config = DistilBertConfig(
    vocab_size=tokenizer.vocab_size,
)

In [17]:
tokenizer.max_model_input_sizes

{'distilbert-base-uncased': 512,
 'distilbert-base-uncased-distilled-squad': 512,
 'distilbert-base-cased': 512,
 'distilbert-base-cased-distilled-squad': 512,
 'distilbert-base-german-cased': 512,
 'distilbert-base-multilingual-cased': 512}

In [16]:
import os
base = "/mnt/transcriber/Call_Scoring/transcriptions/csr_ch/train/"
for file in os.listdir(base):
    token = tokenizer(base+file)
    print(len(token['input_ids']))

IRCall_200133938080200512_singlech.txt
35
IRCall_100162646340200119_singlech.txt
35
IRCall_300147714640210203_singlech.txt
35
IRCall_200165853580200518_singlech.txt
35
IRCall_100197435940200127_singlech.txt
35
IRCall_300154049730200902_singlech.txt
35
IRCall_200148882640210116_singlech.txt
35
IRCall_100158123680200430_singlech.txt
35
IRCall_200184235280200520_singlech.txt
35
IRCall_300131037720191220_singlech.txt
35
IRCall_100128380960200305_singlech.txt
35
IRCall_100125142140200915_singlech.txt
35
IRCall_200140512040200203_singlech.txt
35
IRCall_1001939627A0200627_singlech.txt
35
IRCall_200173220120201009_singlech.txt
35
IRCall_2001720035A0191207_singlech.txt
35
IRCall_100187038080190503_singlech.txt
35
IRCall_200184004320201012_singlech.txt
35
IRCall_300132471530210320_singlech.txt
35
IRCall_200161810240200206_singlech.txt
35
IRCall_300160505140210205_singlech.txt
35
IRCall_200124681860200323_singlech.txt
35
IRCall_100173529030210105_singlech.txt
35
IRCall_100166751360200311_singlech

In [14]:
token

{'input_ids': [0, 2, 22, 16939, 2, 15827, 158, 2, 264, 9, 4267, 2, 21706, 615, 2, 12, 39, 53, 9, 268, 2, 2828, 2, 4483, 13574, 51, 9, 2, 9, 801, 275, 7, 29, 660, 4], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}